# Max-Cut with QAOA (CUDA-Q)

QAOA template for Max-Cut using CUDA-Q. We build a cost Hamiltonian,
define a parameterized QAOA kernel, and time a simple classical
optimization of the expectation value.


In [ ]:
import time
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import cudaq


In [ ]:
n_nodes = 6
G = nx.erdos_renyi_graph(n_nodes, 0.5, seed=1)
pos = nx.spring_layout(G, seed=1)
nx.draw(G, pos, with_labels=True)
plt.title("Max-Cut instance (CUDA-Q)")
plt.show()


In [ ]:
def maxcut_spinop(G):
    H = cudaq.SpinOperator()
    for i, j in G.edges():
        H += 1.0 * cudaq.spin.z(i) * cudaq.spin.z(j)
    return H

H = maxcut_spinop(G)
n = len(G.nodes())


In [ ]:
@cudaq.kernel
def qaoa_kernel(num_qubits: int, gammas: list[float], betas: list[float]):
    qs = cudaq.qvector(num_qubits)
    for i in range(num_qubits):
        cudaq.h(qs[i])

    for layer in range(len(gammas)):
        gamma = gammas[layer]
        beta = betas[layer]

        # Cost layer
        for (i, j) in G.edges():
            cudaq.exp_i_z_z(qs[i], qs[j], 2 * gamma)

        # Mixer layer
        for i in range(num_qubits):
            cudaq.rx(qs[i], 2 * beta)


In [ ]:
def expected_value(params):
    p = len(params) // 2
    gammas = params[:p]
    betas = params[p:]
    result = cudaq.observe(qaoa_kernel, H, n, gammas, betas)
    return result.expectation()


In [ ]:
p = 2
x0 = np.random.uniform(0, np.pi, 2 * p)

opt = cudaq.optimizers.NelderMead()

start = time.perf_counter()
res = opt.optimize(2 * p, expected_value, x0)
end = time.perf_counter() if hasattr(time, "perfCounter") else time.perf_counter()

print("Optimal parameters:", res.optimal_value)
print("Optimal energy:", res.optimal_cost)
print(f"Runtime: {end - start:.4f} s")
